In [1]:
import praw
import pandas as pd
import datetime as dt
from praw.models import MoreComments

from bs4 import BeautifulSoup
import requests
import itertools
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
url='https://redditmetrics.com/top'
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
table = soup.find("table")
rows = table.find_all("tr")
top_subs = []
for row in rows:
        cells = row.find_all("td")
        cells = cells[0:11]
        top_subs.append([cell.text for cell in cells])

top_subs=pd.DataFrame(top_subs)
top_subs.columns=['Rank','Subreddit','Subscribers']
top_subs=top_subs.drop(0)
top_subs=top_subs.reset_index(drop=True)
i=0
for i in range(len(top_subs['Rank'])):
    top_subs['Subreddit'][i]=top_subs['Subreddit'][i][3:]
    i=+1

csv = top_subs
export_csv = csv.to_csv (r'~/Downloads/top_reddit.csv', index = None, header=True)
print (export_csv)

None


In [18]:
# Add your details
reddit = praw.Reddit(client_id='', \
                     client_secret='', \
                     user_agent='', \
                     username='', \ 
                     password='')


In [27]:
topics_data=pd.DataFrame([])
a=0
for a in range(len(top_subs['Rank'])):
    print(a)
    target=top_subs['Subreddit'][a]
    subreddit = reddit.subreddit(target)
    cont_subreddit = subreddit.controversial()
    cont_subreddit = subreddit.controversial(limit=500)
    topics_dict = { "title":[], \
                "score":[], \
                "id":[], "url":[], \
                "comms_num": [], \
                "created": [], \
                "body":[]}
    for submission in cont_subreddit:
        topics_dict["title"].append(submission.title)
        topics_dict["score"].append(submission.score)
        topics_dict["id"].append(submission.id)
        topics_dict["url"].append(submission.url)
        topics_dict["comms_num"].append(submission.num_comments)
        topics_dict["created"].append(submission.created)
        topics_dict["body"].append(submission.selftext)
    interim=pd.DataFrame(topics_dict)
    interim['subreddit']=interim['title']
    b=0
    for b in range(len(interim['subreddit'])):
        interim['subreddit'][b]=target
        b=+1
    topics_data = topics_data.append(interim)
    
    a=+1
topics_data=topics_data.reset_index(drop=True)
csv = pd.DataFrame(topics_data)
export_csv = csv.to_csv (r'~/Downloads/topics_1_cont.csv', index = None, header=True)
print (export_csv)

0


C:\Users\kevin.sweeney\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
None


In [28]:
def get_date(created):
        return dt.datetime.fromtimestamp(created)
        _timestamp = topics_data["created"].apply(get_date)
        topics_data = topics_data.assign(timestamp = _timestamp)
topics_data['comments']=topics_data['title']
topics_data['top_comments']=topics_data['title']


In [ ]:
start=time.time()
i=2700
i_int=i-1
while i <= len(topics_data['id']):
    if i>0 and i%100==0:
        safe=topics_data
        end=time.time()
        now=end-start
        per_it=now/(i-i_int)
        remaining=len(topics_data['id'])-i
        time_remaining = (remaining *per_it)/3600
        print('Done:',i,'Time per post:',per_it, 'Estimated Time Remaining:',time_remaining,'hours')
        csv = pd.DataFrame(topics_data)
        export_csv = csv.to_csv (r'~/Downloads/topics_safety_cont.csv', index = None, header=True)
        print (export_csv)
    post=topics_data['id'][i]
    submission = reddit.submission(id=post)
    submission.comments.replace_more(limit=0)
    txt=""
    for comment in submission.comments.list():
        txt = txt+comment.body
    topics_data['comments'][i]=txt
#     txt_2=""
#     for top_level_comment in submission.comments.list():
#         txt_2 = txt_2+top_level_comment.body
#     topics_data['top_comments'][i]=txt_2
    i=i+1
csv = pd.DataFrame(topics_data)
export_csv = csv.to_csv (r'~/Downloads/topics_2_cont.csv', index = None, header=True)
print (export_csv)

Done: 2700 Time per post: 0.0009613037109375 Estimated Time Remaining: 0.012518043518066407 hours
None


C:\Users\kevin.sweeney\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Done: 2800 Time per post: 3.695498435804159 Estimated Time Remaining: 48.01992259124521 hours
None
Done: 2900 Time per post: 3.6540444988516434 Estimated Time Remaining: 47.37976198941551 hours
None
Done: 3000 Time per post: 3.3892027904028907 Estimated Time Remaining: 43.85157688171562 hours
None
Done: 3100 Time per post: 3.2806387696777497 Estimated Time Remaining: 42.35578038218115 hours
None
Done: 3200 Time per post: 3.153720886645441 Estimated Time Remaining: 40.629561389369144 hours
None
Done: 3300 Time per post: 3.0011798446864733 Estimated Time Remaining: 38.58100056451258 hours
None
Done: 3400 Time per post: 2.8926707640524087 Estimated Time Remaining: 37.10573422588228 hours
None
Done: 3500 Time per post: 2.7719936311319375 Estimated Time Remaining: 35.48074848025793 hours
None
Done: 3600 Time per post: 2.6331456742196715 Estimated Time Remaining: 33.630390265262854 hours
None
Done: 3700 Time per post: 2.630870617352999 Estimated Time Remaining: 33.52825362598285 hours
None
D

In [13]:
csv = pd.DataFrame(topics_data)
export_csv = csv.to_csv (r'~/Downloads/topics_2.csv', index = None, header=True)
print (export_csv)

None


In [87]:
topics_data_recover = pd.read_csv (r'~/Downloads/topics_2.csv')